<a href="https://colab.research.google.com/github/06-preetinder/ai-waste-classification/blob/main/1_data_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [26]:
import zipfile

zip_path = '/content/drive/MyDrive/ai-project/dataset.zip'
extract_path = '/content/'

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)
import shutil
shutil.rmtree('/content/dataset/trash')


In [27]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Set image size and batch size
IMAGE_SIZE = (224, 224)
BATCH_SIZE = 32
DATASET_PATH = '/content/dataset'  # Change if your path is different

# Train and validation generators with strong augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)

train_generator = train_datagen.flow_from_directory(
    DATASET_PATH,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training',
    shuffle=True
)

val_generator = train_datagen.flow_from_directory(
    DATASET_PATH,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation',
    shuffle=True
)

# Save class names
class_names = list(train_generator.class_indices.keys())


Found 1914 images belonging to 5 classes.
Found 476 images belonging to 5 classes.


In [28]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    MaxPooling2D(2, 2),

    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),

    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),

    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.4),
    Dense(len(class_names), activation='softmax')
])

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_3 (Conv2D)               │ (None, 222, 222, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 111, 111, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 109, 109, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 54, 54, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 52, 52, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 26, 26, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 86528)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 256)            │    22,151,424 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 5)              │         1,285 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 22,245,957 (84.86 MB)

 Trainable params: 22,245,957 (84.86 MB)

 Non-trainable params: 0 (0.00 B)

In [29]:
from sklearn.utils import class_weight
import numpy as np

# Get the class indices from the generator
class_indices = train_generator.class_indices
classes = list(class_indices.keys())

# Get class labels for each image
labels = train_generator.classes

# Compute weights
weights = class_weight.compute_class_weight(
    class_weight='balanced',
    classes=np.unique(labels),
    y=labels
)

# Convert to dictionary: {class_index: weight}
class_weights = dict(enumerate(weights))

print("Class Weights:", class_weights)


Class Weights: {0: np.float64(1.1851393188854489), 1: np.float64(0.9546134663341646), 2: np.float64(1.1670731707317072), 3: np.float64(0.8042016806722689), 4: np.float64(0.9917098445595854)}


In [30]:
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=20,
    callbacks=[early_stop],
    class_weight=class_weights  # Still helps!
)


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
60/60 ━━━━━━━━━━━━━━━━━━━━ 42s 651ms/step - accuracy: 0.3270 - loss: 1.5944 - val_accuracy: 0.3634 - val_loss: 1.5100
Epoch 2/20
60/60 ━━━━━━━━━━━━━━━━━━━━ 32s 528ms/step - accuracy: 0.4142 - loss: 1.2870 - val_accuracy: 0.3761 - val_loss: 1.3513
Epoch 3/20
60/60 ━━━━━━━━━━━━━━━━━━━━ 33s 549ms/step - accuracy: 0.4876 - loss: 1.1998 - val_accuracy: 0.5294 - val_loss: 1.2078
Epoch 4/20
60/60 ━━━━━━━━━━━━━━━━━━━━ 31s 515ms/step - accuracy: 0.5386 - loss: 1.1135 - val_accuracy: 0.4874 - val_loss: 1.2327
Epoch 5/20
60/60 ━━━━━━━━━━━━━━━━━━━━ 31s 520ms/step - accuracy: 0.5480 - loss: 1.0756 - val_accuracy: 0.4664 - val_loss: 1.2151
Epoch 6/20
60/60 ━━━━━━━━━━━━━━━━━━━━ 37s 613ms/step - accuracy: 0.5888 - loss: 1.0410 - val_accuracy: 0.5105 - val_loss: 1.1727
Epoch 7/20
60/60 ━━━━━━━━━━━━━━━━━━━━ 32s 529ms/step - accuracy: 0.5894 - loss: 0.9770 - val_accuracy: 0.4958 - val_loss: 1.1548
Epoch 8/20
60/60 ━━━━━━━━━━━━━━━━━━━━ 33s 552ms/step - accuracy: 0.6244 - loss: 0.9256 - val_accu

In [31]:
from tensorflow.keras.callbacks import EarlyStopping

early_stop = EarlyStopping(
    patience=5,               # wait for 5 epochs without improvement
    restore_best_weights=True
)


In [32]:
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=30,                # let it go longer
    callbacks=[early_stop],
    class_weight=class_weights
)


Epoch 1/30
60/60 ━━━━━━━━━━━━━━━━━━━━ 33s 551ms/step - accuracy: 0.7199 - loss: 0.7798 - val_accuracy: 0.5987 - val_loss: 1.1127
Epoch 2/30
60/60 ━━━━━━━━━━━━━━━━━━━━ 34s 573ms/step - accuracy: 0.7033 - loss: 0.8152 - val_accuracy: 0.5882 - val_loss: 1.0370
Epoch 3/30
60/60 ━━━━━━━━━━━━━━━━━━━━ 32s 536ms/step - accuracy: 0.6702 - loss: 0.8603 - val_accuracy: 0.5693 - val_loss: 1.1356
Epoch 4/30
60/60 ━━━━━━━━━━━━━━━━━━━━ 40s 530ms/step - accuracy: 0.6878 - loss: 0.8236 - val_accuracy: 0.5546 - val_loss: 1.2283
Epoch 5/30
60/60 ━━━━━━━━━━━━━━━━━━━━ 37s 616ms/step - accuracy: 0.6525 - loss: 0.8835 - val_accuracy: 0.5861 - val_loss: 1.0545
Epoch 6/30
60/60 ━━━━━━━━━━━━━━━━━━━━ 32s 535ms/step - accuracy: 0.7241 - loss: 0.7556 - val_accuracy: 0.5966 - val_loss: 1.1183
Epoch 7/30
60/60 ━━━━━━━━━━━━━━━━━━━━ 34s 567ms/step - accuracy: 0.7102 - loss: 0.7848 - val_accuracy: 0.6429 - val_loss: 0.9481
Epoch 8/30
60/60 ━━━━━━━━━━━━━━━━━━━━ 32s 534ms/step - accuracy: 0.7449 - loss: 0.7077 - val_accu

In [33]:
model.save("waste_classification_model.h5")